In [14]:




def query_data(
        query, 
        
        timpespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]]):
    response = client.query_workspace(Alpine, query, timespan=timpespan, include_statistics=True)
    print(response.statistics)
    # get total bytes processed
    # print(response.statistics["network"]['interClusterTotalBytes'])
    return response
    if response.status == LogsQueryStatus.SUCCESS:
        return response.tables
    else:
        return response.partial_data
    
    
# 45 days
start_time = datetime(2024, 6, 18, 0, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 8, 2, 0, 0, 0, 0, tzinfo=timezone.utc)

# for 3 hours
duration = timedelta(hours=3)
# end_time = start_time + duration

resutl = query_data("AADManagedIdentitySignInLogs", (start_time, duration))
print(resutl)

{'query': {'executionTime': 0.0156209, 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 35587, 'missbytes': 0, 'retrievebytes': 0}, 'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0}, 'bypassbytes': 0}}, 'cpu': {'user': '00:00:00', 'kernel': '00:00:00', 'totalCpu': '00:00:00', 'breakdown': {'queryExecution': '00:00:00', 'queryPlanning': '00:00:00'}}, 'memory': {'peakPerNode': 65546752}, 'network': {'interClusterTotalBytes': 12882, 'crossClusterTotalBytes': 0}}, 'inputDatasetStatistics': {'extents': {'total': 68, 'scanned': 1, 'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z', 'scannedMaxDatetime': '2024-06-18T00:00:00.0000000Z'}, 'rows': {'total': 43242, 'scanned': 215}, 'rowstores': {'scannedRows': 611, 'scannedValuesSize': 2061095}, 'shards': {'queriesGeneric': 1, 'queriesSpecialized': 0}}, 'datasetStatistics': [{'tableRowCount': 1, 'tableSize': 603}], 'crossClusterResourceUsage': {}}}


In [62]:
import os
import pandas as pd
import json
from datetime import timedelta, datetime, timezone
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.core.exceptions import HttpResponseError
from azure.identity import DefaultAzureCredential
from typing import Optional, Union, Tuple
from textwrap import dedent

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)
def get_count(
        workspace_id: str,
        table_name: str,
        timpespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]]
    ) -> int:

    response = client.query_workspace(workspace_id, f"{table_name} | count", timespan=timpespan)

    if response.status == LogsQueryStatus.SUCCESS:
        return response.tables[0].rows[0][0]
    else:
        return -1
    
def calculate_size_per_entry(
        workspace_id: str,
        table_name: str,
        timpespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]]
    ) -> int:

    response = client.query_workspace(workspace_id, f"{table_name} | limit 2000", timespan=timpespan, include_statistics=True)
    if response.status == LogsQueryStatus.SUCCESS:
        if  response.statistics['query']['datasetStatistics'][0]['tableRowCount'] == 0:
            print(f"Table {table_name} is empty.")
            return 0
        return response.statistics['query']['datasetStatistics'][0]['tableSize'] / response.statistics['query']['datasetStatistics'][0]['tableRowCount']
    else:
        return -1
    

def check_segemented_query(
        workspace_id: str,
        table_name: str,
        timespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]],
        max_size_allowed: int = 60000000, # in bytes, ~60MB
        max_count_allowed: int = 500000,
        verbose: bool = False
    ) -> Tuple[bool, int]:
    """
    Check if the query need to be segmented or not
    
    Args:
    - workspace_id: Azure Log Analytics workspace id
    - table_name: table name to query
    - timespan: time range to query
    - max_size_allowed: maximum size allowed for a single query
    - max_count_allowed: maximum count allowed for a single query

    Returns:
    - Tuple[bool, int, int, int]:
        - bool: if the query need to be segmented or not
        - int: number of row per query
        - int: total count of the table
        - int: total size of the table, in bytes
    """
    # https://learn.microsoft.com/en-us/azure/azure-monitor/service-limits#la-query-api

    total_count = get_count(workspace_id, table_name, timespan)
    size_per_entry = calculate_size_per_entry(workspace_id, table_name, timespan)
    if total_count == -1 or size_per_entry == -1:
        return True, 0, -1, -1
    total_size = total_count * size_per_entry 
    if total_size < max_size_allowed and total_count < max_count_allowed:
        if verbose:
            print(f"Total count: {total_count}", f"Size per entry: {size_per_entry}", f"Total size: {total_size/1024/1024} MB")
        return False, -1, total_count, total_size
    
    row_per_query = max_count_allowed
    if total_size > max_size_allowed:
        # get number of row per query 
        row_per_query = min(row_per_query, max_size_allowed // size_per_entry)
    
    if verbose:
        print(f"Table: {table_name}", 
              f"Total count: {total_count}", 
              f"Size per entry: {size_per_entry} Bytes", 
              f"Total size: {total_size/1024/1024} MB", 
              f"Row per query: {row_per_query}", 
              f"Estimate table count: {total_count // row_per_query}",
              sep=", ",
              )
    return True, int(row_per_query), total_count, total_size

def query_and_save_data(
        workspace_id: str,
        table_name: str,
        timespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]],
        file_path: str,
        verbose: bool = False
):
    
    def save_table(file_path_name, response, need_metadata=False):
        table = response.tables[0]
        df = pd.DataFrame(data=table.rows, columns=table.columns)
        if len(df) == 0:
            print(f"Table {table_name} is empty. Skipping.")
            return -1, -1
        metadata = dict(zip(df.columns.tolist(), table.columns_types))
        if need_metadata:
            with open(file_path_name.replace(".csv", "_metadata.json"), "w") as f:
                json.dump(metadata, f)

        json_columns = [col for col, dtype in metadata.items() if dtype == "dynamic"]
        for column in json_columns:
            df[column] = df[column].apply(lambda x: "{}" if x == "" else x)
            df[column] = df[column].apply(lambda x: "{}" if pd.isnull(x) else x)
        
        df.to_csv(file_path_name, index=False, sep="¤", encoding='utf-8')
        return df["TimeGenerated"].min(), df["TimeGenerated"].max()
    
    max_size_allowed: int = 60000000
    need_segement, row_per_query, total_count, total_size = check_segemented_query(workspace_id, table_name, timespan, verbose=verbose, max_size_allowed=max_size_allowed)

    if need_segement:
        updated_file_path = os.path.join(file_path, table_name)
        os.makedirs(updated_file_path, exist_ok=True)
        
        # get total time range in hours, conver to start and end time
        if isinstance(timespan, timedelta):
            timespan = (datetime.utcnow() - timespan, datetime.utcnow())
        elif isinstance(timespan[1], timedelta):
            total_hours = timespan[1].total_seconds() / 3600
            timespan = (timespan[0], timespan[0] + timespan[1])
        else:
            total_hours = (timespan[1] - timespan[0]).total_seconds() / 3600
        
        # print(type(timespan[0]))
        # get size per hour
        size_per_hour = total_size / total_hours
        # approximate 200MB per time chunk
        time_chunk = int(200 * 1024 * 1024 / size_per_hour)
        print(f"Time chunk: {time_chunk} hours")

        tmp_start_time = timespan[0]
        chunk_id = 0
        # if table_name == "AADNonInteractiveUserSignInLogs":
        #     # 2024-07-15 13:02:04.923788+00:00
        #     tmp_start_time = datetime(2024, 7, 15, 13, 2, 4, 923788, tzinfo=timezone.utc) + timedelta(milliseconds=1)
        #     chunk_id = 44
        #     print(f"Resuming from chunk 44, {tmp_start_time}")
        query_template = dedent("""{table_name} 
| order by TimeGenerated asc
| serialize
| extend rn = row_number() 
| where rn <= {end}
""")        
        while tmp_start_time < timespan[1]:
            # get data from a time chunk
            tmp_timespan = (tmp_start_time, min(tmp_start_time + timedelta(hours=time_chunk), timespan[1]))

            response = client.query_workspace(
                workspace_id, 
                query_template.format(table_name=table_name, end=row_per_query),
                timespan=tmp_timespan
            ) # only return the first row_per_query rows
    
            if response.status != LogsQueryStatus.SUCCESS:
                error = response.partial_error
                print(f"Getting error, retry chunk {chunk_id}", error)
                # update max size allowed and row per query
                max_size_allowed -= 2000000
                row_per_query = min(row_per_query, max_size_allowed // (total_size/total_count))
            else:
                # set new start time to be the last time from the response
                earliest, latest = save_table(os.path.join(updated_file_path, f"{table_name}_{chunk_id}.csv"), response, need_metadata=chunk_id == 0)
                if earliest == -1:
                    print(f"Reached end of the table. Exiting.")
                    break
                # from pd timestamp to datetime.datetime, and +1 in milliseconds
                tmp_start_time = latest.to_pydatetime() + timedelta(milliseconds=1)
                chunk_id += 1
                print(f"Chunk {chunk_id}: {earliest} - {latest}")  #Required span: {tmp_timespan} || Actual:
            
    else:
        response = client.query_workspace(workspace_id, f"{table_name}", timespan=timespan)
        save_table(os.path.join(file_path, f"{table_name}.csv"), response, need_metadata=True)





In [ ]:
        # # number of queries to run
        # query_count = int(total_count // row_per_query) + 1

        # for i in range(query_count):
        #     start = i * row_per_query
        #     end = min((i + 1) * row_per_query, total_count)
        #     query = query_template.format(table_name=table_name, start=start, end=end)
        #     response = client.query_workspace(workspace_id, query, timespan=timespan)

        #     if response.status != LogsQueryStatus.SUCCESS:
        #         error = response.partial_error
        #         data = response.partial_data
        #         print(error)
        #         raise HttpResponseError(response.error)

        #     save_table(os.path.join(updated_file_path, f"{table_name}_{i}.csv"), response, need_metadata=i==1)

In [59]:
ATEVET_17 = "0fbd2874-9307-4572-b499-f8fa3cc75daf"
Alpine = "e34d562e-ef12-4c4e-9bc0-7c6ae357c015"
# 45 days
start_time = datetime(2024, 6, 18, 0, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 8, 2, 0, 0, 0, 0, tzinfo=timezone.utc)

# for 3 hours
duration = timedelta(hours=3)
# end_time = start_time + duration

from download_logs import LIST_TABLES
# # print table size of each table and total size
total_size = 0
for table in LIST_TABLES:
    need_segement, row_per_query, total_count, total_size_table = check_segemented_query(Alpine, table, (start_time, end_time))
    
    if total_count == -1 or total_size_table == -1:
        print(f"Table {table} is failed to get size.")
        continue

    total_size += total_size_table
    print(f"Table {table} has {total_count} rows and {round(total_size_table/1024/1024, 3)} MB   (GB: {round(total_size_table/1024/1024/1024, 3)})")

print(f"Total size: {round(total_size/1024/1024, 3)} MB   (GB: {round(total_size/1024/1024/1024, 3)})")

Table AADManagedIdentitySignInLogs has 24516 rows and 12.774 MB   (GB: 0.012)
Table AADNonInteractiveUserSignInLogs has 1627812 rows and 7060.33 MB   (GB: 6.895)
Table AADProvisioningLogs has 635 rows and 1.582 MB   (GB: 0.002)
Table AADRiskyUsers has 175 rows and 0.034 MB   (GB: 0.0)
Table AADServicePrincipalSignInLogs has 190271 rows and 146.289 MB   (GB: 0.143)
Table AADUserRiskEvents has 336 rows and 0.296 MB   (GB: 0.0)
Table Alert has 16 rows and 0.022 MB   (GB: 0.0)
Table AmlDataLabelEvent has 1 rows and 0.0 MB   (GB: 0.0)
Table AmlDataStoreEvent has 12 rows and 0.005 MB   (GB: 0.0)
Table AuditLogs has 73633 rows and 137.771 MB   (GB: 0.135)
Table AZFWApplicationRule has 222692 rows and 72.146 MB   (GB: 0.07)
Table AZFWApplicationRuleAggregation has 49038 rows and 17.156 MB   (GB: 0.017)
Table AZFWDnsQuery has 461803 rows and 130.201 MB   (GB: 0.127)
Table AZFWFlowTrace has 86824 rows and 21.609 MB   (GB: 0.021)
Table AZFWIdpsSignature has 8221 rows and 2.49 MB   (GB: 0.002)
Tab

In [42]:
from datetime import datetime

# Your datetime string
datetime_str = "2024-06-18 00:01:18.645615+00:00"

# Convert the string to a datetime object
dt_obj = datetime.fromisoformat(datetime_str)

# Print the datetime object
print(dt_obj)
print(dt_obj+timedelta(microseconds=1))
print(dt_obj+timedelta(milliseconds=1))

2024-06-18 00:01:18.645615+00:00
2024-06-18 00:01:18.645616+00:00
2024-06-18 00:01:18.646615+00:00


In [64]:
file_path = "data/alpineSkiHouse"
os.makedirs(file_path, exist_ok=True)
for table in LIST_TABLES:
    if os.path.exists(os.path.join(file_path, f"{table}.csv")) or os.path.exists(os.path.join(file_path, table)):
        print(f"Table {table} is already saved.")
        continue
    try :
        query_and_save_data(Alpine, table, (start_time, end_time), file_path, verbose=True)
        print(f"Table {table} is saved.")
    except HttpResponseError as e:
        print(f"Table {table} is failed to save.")
        print(e)
        continue

Table AADManagedIdentitySignInLogs is already saved.
Table AADNonInteractiveUserSignInLogs is already saved.
Table AADProvisioningLogs is already saved.
Table AADRiskyUsers is already saved.
Table AADServicePrincipalSignInLogs is already saved.
Table AADUserRiskEvents is already saved.
Table Alert is already saved.
Table AmlDataLabelEvent is already saved.
Table AmlDataStoreEvent is already saved.
Table AuditLogs is already saved.
Table AZFWApplicationRule is already saved.
Table AZFWApplicationRuleAggregation is already saved.
Table AZFWDnsQuery is already saved.
Table AZFWFlowTrace is already saved.
Table AZFWIdpsSignature is already saved.
Table AZFWNatRule is already saved.
Table AZFWNatRuleAggregation is already saved.
Table AZFWNetworkRule is already saved.
Table AZFWNetworkRuleAggregation is already saved.
Table AZFWThreatIntel is already saved.
Table AzureActivity is already saved.
Table AzureMetrics is already saved.
Table ContainerRegistryRepositoryEvents is already saved.
Ta

In [4]:
6882/1024

6.720703125

In [57]:
for i in range(0, 4, 3):
    print(i)

0
3


In [55]:
segemented_query(Alpine, "AADManagedIdentitySignInLogs", (start_time, end_time))

Total count: 24516 Size per entry: 558.68 Total size: 13.062094573974608 MB


(False, -1)

In [ ]:
# get_size
resutl = query_data("AADManagedIdentitySignInLogs | count", (start_time, end_time))

In [47]:
duration = timedelta(hours=20)
# end_time = start_time + duration

resutl = query_data("AADManagedIdentitySignInLogs | limit 200", (start_time, end_time))

{'query': {'executionTime': 0.0312273, 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 2142188, 'missbytes': 0, 'retrievebytes': 0}, 'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0}, 'bypassbytes': 0}}, 'cpu': {'user': '00:00:00.0312500', 'kernel': '00:00:00.0156250', 'totalCpu': '00:00:00.0468750', 'breakdown': {'queryExecution': '00:00:00.0468750', 'queryPlanning': '00:00:00'}}, 'memory': {'peakPerNode': 65915136}, 'network': {'interClusterTotalBytes': 2437785, 'crossClusterTotalBytes': 0}}, 'inputDatasetStatistics': {'extents': {'total': 68, 'scanned': 40, 'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z', 'scannedMaxDatetime': '2024-08-02T21:19:30.5753348Z'}, 'rows': {'total': 43242, 'scanned': 25069}, 'rowstores': {'scannedRows': 630, 'scannedValuesSize': 2124963}, 'shards': {'queriesGeneric': 33, 'queriesSpecialized': 0}}, 'datasetStatistics': [{'tableRowCount': 200, 'tableSize': 108997}], 'crossClusterResourceUsage': {}}}


In [44]:
resutl.statistics['query']

{'executionTime': 0.0156376,
 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 187666,
     'missbytes': 0,
     'retrievebytes': 0},
    'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0},
    'bypassbytes': 0}},
  'cpu': {'user': '00:00:00.0625000',
   'kernel': '00:00:00.0468750',
   'totalCpu': '00:00:00.1093750',
   'breakdown': {'queryExecution': '00:00:00.1093750',
    'queryPlanning': '00:00:00'}},
  'memory': {'peakPerNode': 66525440},
  'network': {'interClusterTotalBytes': 7858560, 'crossClusterTotalBytes': 0}},
 'inputDatasetStatistics': {'extents': {'total': 68,
   'scanned': 40,
   'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z',
   'scannedMaxDatetime': '2024-08-02T21:19:30.5753348Z'},
  'rows': {'total': 43242, 'scanned': 25069},
  'rowstores': {'scannedRows': 627, 'scannedValuesSize': 2114383},
  'shards': {'queriesGeneric': 4, 'queriesSpecialized': 0}},
 'datasetStatistics': [{'tableRowCount': 1000, 'tableSize': 547144}],
 'crossClusterResource

In [45]:
7858560 / 547144

14.36287339347594

In [42]:
resutl.statistics['query']

{'executionTime': 0.0468879,
 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 1771040,
     'missbytes': 0,
     'retrievebytes': 0},
    'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0},
    'bypassbytes': 0}},
  'cpu': {'user': '00:00:00.0468750',
   'kernel': '00:00:00.0312500',
   'totalCpu': '00:00:00.0781250',
   'breakdown': {'queryExecution': '00:00:00.0781250',
    'queryPlanning': '00:00:00'}},
  'memory': {'peakPerNode': 65546496},
  'network': {'interClusterTotalBytes': 1267742, 'crossClusterTotalBytes': 0}},
 'inputDatasetStatistics': {'extents': {'total': 68,
   'scanned': 40,
   'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z',
   'scannedMaxDatetime': '2024-08-02T21:19:30.5753348Z'},
  'rows': {'total': 43242, 'scanned': 25069},
  'rowstores': {'scannedRows': 626, 'scannedValuesSize': 2110866},
  'shards': {'queriesGeneric': 31, 'queriesSpecialized': 0}},
 'datasetStatistics': [{'tableRowCount': 100, 'tableSize': 54988}],
 'crossClusterResource

In [49]:
108997/54988

1.9821961155161125

In [48]:
resutl.statistics['query']

{'executionTime': 0.0312273,
 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 2142188,
     'missbytes': 0,
     'retrievebytes': 0},
    'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0},
    'bypassbytes': 0}},
  'cpu': {'user': '00:00:00.0312500',
   'kernel': '00:00:00.0156250',
   'totalCpu': '00:00:00.0468750',
   'breakdown': {'queryExecution': '00:00:00.0468750',
    'queryPlanning': '00:00:00'}},
  'memory': {'peakPerNode': 65915136},
  'network': {'interClusterTotalBytes': 2437785, 'crossClusterTotalBytes': 0}},
 'inputDatasetStatistics': {'extents': {'total': 68,
   'scanned': 40,
   'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z',
   'scannedMaxDatetime': '2024-08-02T21:19:30.5753348Z'},
  'rows': {'total': 43242, 'scanned': 25069},
  'rowstores': {'scannedRows': 630, 'scannedValuesSize': 2124963},
  'shards': {'queriesGeneric': 33, 'queriesSpecialized': 0}},
 'datasetStatistics': [{'tableRowCount': 200, 'tableSize': 108997}],
 'crossClusterResourc

In [ ]:
['resourceUsage']['network']['interClusterTotalBytes']

In [32]:
import pandas as pd
for k in resutl.tables:
    m = pd.DataFrame(k.rows, columns=k.columns)
    print(m)

   Count
0  24516


In [21]:
resutl.statistics['query']['resourceUsage']['network']['interClusterTotalBytes']
# bytes to mib
# resutl.statistics['query']['resourceUsage']['network']['interClusterTotalBytes'] / 1024 / 1024

12882

In [13]:

ATEVET_17 = "0fbd2874-9307-4572-b499-f8fa3cc75daf"
Alpine = "e34d562e-ef12-4c4e-9bc0-7c6ae357c015"
def query_data(query, timpespan: Optional[Union[timedelta, Tuple[datetime, timedelta], Tuple[datetime, datetime]]]):
    response = client.query_workspace(Alpine, query, timespan=timpespan, include_statistics=True)
    print(response.statistics)
    # get total bytes processed
    print(response.statistics["network"]['interClusterTotalBytes'])
    if response.status == LogsQueryStatus.SUCCESS:
        return response.tables
    else:
        return response.partial_data
    
    
# 45 days
start_time = datetime(2024, 6, 18, 0, 0, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 8, 2, 0, 0, 0, 0, tzinfo=timezone.utc)

# for 3 hours
duration = timedelta(hours=3)
# end_time = start_time + duration

resutl = query_data("AADManagedIdentitySignInLogs", (start_time, duration))
print(resutl)

{'query': {'executionTime': 0.0156209, 'resourceUsage': {'cache': {'shards': {'hot': {'hitbytes': 35587, 'missbytes': 0, 'retrievebytes': 0}, 'cold': {'hitbytes': 0, 'missbytes': 0, 'retrievebytes': 0}, 'bypassbytes': 0}}, 'cpu': {'user': '00:00:00', 'kernel': '00:00:00', 'totalCpu': '00:00:00', 'breakdown': {'queryExecution': '00:00:00', 'queryPlanning': '00:00:00'}}, 'memory': {'peakPerNode': 65546752}, 'network': {'interClusterTotalBytes': 12882, 'crossClusterTotalBytes': 0}}, 'inputDatasetStatistics': {'extents': {'total': 68, 'scanned': 1, 'scannedMinDatetime': '2024-06-18T00:00:00.0000000Z', 'scannedMaxDatetime': '2024-06-18T00:00:00.0000000Z'}, 'rows': {'total': 43242, 'scanned': 215}, 'rowstores': {'scannedRows': 611, 'scannedValuesSize': 2061095}, 'shards': {'queriesGeneric': 1, 'queriesSpecialized': 0}}, 'datasetStatistics': [{'tableRowCount': 1, 'tableSize': 603}], 'crossClusterResourceUsage': {}}}


KeyError: 'network'

In [9]:
# get stats
stats = resutl[0].to_dict()

AttributeError: 'LogsTable' object has no attribute 'to_dict'